In [2]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:

#load csv file
import pandas as pd # Import the pandas library with the alias 'pd'
df = pd.read_csv('/content/articles.csv',encoding ='cp1252')
df.head(2)

,Id,Heading,Article.Banner.Image,Outlets,Article.Description,Full_Article,Article_Type,Tonality
0,d6995462-5e87-453b-b64d-e9f1df6e94d2,"A Puzzling Maneuver, Then Freefall: NTSB Repor...",NaN,Essex Caller,<p>The helicopter that crashed in Southeast Al...,<p>The helicopter that crashed in Southeast Al...,Commercial,Negative
1,8b05e939-a89e-4548-b92b-013822e8ee7d,Bell’s Nexus Air Taxi Concept Rings Changes Fo...,NaN,Aviation Week Network,<p>A year after teasing the fledgling electric...,<p>A year after teasing the fledgling electric...,Commercial,Positive


In [6]:
# Assuming the CSV has columns 'Full Article' and 'Article Type'
def preprocess_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML tags
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove special characters and numbers
    text = text.lower()  # Convert to lowercase
    text = text.split()
    text = [word for word in text if word not in stopwords.words('english')]
    return ' '.join(text)

In [7]:
# Apply preprocessing to the 'Full Article' column
df['clean_text'] = df['Full_Article'].apply(preprocess_text)
df.head()

,Id,Heading,Article.Banner.Image,Outlets,Article.Description,Full_Article,Article_Type,Tonality,clean_text
0,d6995462-5e87-453b-b64d-e9f1df6e94d2,"A Puzzling Maneuver, Then Freefall: NTSB Repor...",NaN,Essex Caller,<p>The helicopter that crashed in Southeast Al...,<p>The helicopter that crashed in Southeast Al...,Commercial,Negative,helicopter crashed southeast alaska late septe...
1,8b05e939-a89e-4548-b92b-013822e8ee7d,Bell’s Nexus Air Taxi Concept Rings Changes Fo...,NaN,Aviation Week Network,<p>A year after teasing the fledgling electric...,<p>A year after teasing the fledgling electric...,Commercial,Positive,year teasing fledgling electric vertical takeo...
2,69fcd400-bceb-4255-8277-619f2d68ac0b,Bell Helicopter Show Air Taxi Nexus,http://images.tmtpost.com/uploads/images/2019/...,TMTPost,<p>Bell released the full-size design of the v...,<p>Bell released the full-size design of the v...,Commercial,Positive,bell released full size design vertical take l...
3,17943578-c11b-414b-b3f5-063d3a93157b,BELL DÉVOILE LA CONCEPTION INTÉGRALE DE SON TA...,http://www.fredzone.org/wp-content/uploads/201...,Fredzone,<p>Bell est une soci&eacute;t&eacute; am&eacut...,<p>Bell est une soci&eacute;t&eacute; am&eacut...,Commercial,Positive,bell est une soci ricaine dirig e par mitch sn...
4,f33c7b11-5f77-4a98-bb2e-d36689042aea,Les premiers retours d’Olivier Ezratty,NaN,FrenchWeb,<p>It was still anecdotal to observe the explo...,<p>It was still anecdotal to observe the explo...,Commercial,Positive,still anecdotal observe explosion passenger dr...


In [24]:
from sklearn.preprocessing import LabelEncoder



# Fill missing values in 'Tonality' column and encode labels
df['Tonality'] = df['Tonality'].fillna('Unknown')
le = LabelEncoder()
df['Tonality'] = le.fit_transform(df['Tonality'])  # Convert labels to numbers

# Check for NaNs in embeddings
print("NaNs in embeddings:", np.isnan(X).any())

# Prepare the data for training
X = np.array(df['embeddings'].tolist())
y = df['Tonality']

NaNs in embeddings: False


In [9]:
!pip install sentence_transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [10]:
from sentence_transformers import SentenceTransformer

# Load pre-trained SentenceBERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Vectorize the cleaned text
df['embeddings'] = df['clean_text'].apply(lambda x: model.encode(x))

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import numpy as np


In [14]:
# Prepare the data for training
X = np.array(df['embeddings'].tolist())  # Convert embeddings to array
y = df['Tonality']  # Target column

In [25]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
# Initialize and train a classifier (SVM in this case)
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

SVC(kernel='linear')

In [28]:
# Predict on the test set
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.90      0.85        61
           1       0.42      0.37      0.39        54
           2       0.85      0.93      0.89       651
           3       0.41      0.12      0.18        95

    accuracy                           0.80       861
   macro avg       0.62      0.58      0.58       861
weighted avg       0.77      0.80      0.77       861



In [ ]:
# Testing new models for improve accuracy

In [30]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)


In [31]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30]
}
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
print("Best parameters found: ", grid_search.best_params_)


Best parameters found:  {'max_depth': 20, 'n_estimators': 50}


In [33]:
# Best model
best_clf = grid_search.best_estimator_

In [32]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Cross-Validation Scores: ", scores)
print("Mean Accuracy: ", scores.mean())


Cross-Validation Scores:  [0.808418   0.81712627 0.81567489 0.80261248 0.81831395]
Mean Accuracy:  0.8124291187092855


In [34]:
best_clf.fit(X_train, y_train)
y_pred = best_clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.62      0.74        61
           1       1.00      0.07      0.14        54
           2       0.80      1.00      0.89       651
           3       1.00      0.04      0.08        95

    accuracy                           0.81       861
   macro avg       0.93      0.43      0.46       861
weighted avg       0.84      0.81      0.74       861



In [35]:
# Save the model
joblib.dump(best_clf, 'tonality_classifier.pkl')


['tonality_classifier.pkl']

In [36]:
# Load the model
loaded_model = joblib.load('tonality_classifier.pkl')

In [50]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/unknown_articles.csv')


In [52]:
df.head()

,Article.URL
0,http://australianaviation.com.au/2018/10/a-com...
1,http://australianaviation.com.au/2018/10/victo...
2,http://australianaviation.com.au/2018/10/army-...
3,https://attain.news/community/special-sea-king...
4,https://m.ariva.de/amp/ad-hoc-airbus-board-of-...


In [ ]:
print("Predictions have been saved to 'predictions.csv'.")